In [1]:
import numpy as np
import pandas as pd
import QUANTAXIS as qa

QUANTAXIS>> start QUANTAXIS
QUANTAXIS>> Welcome to QUANTAXIS, the Version is 1.0.41
QUANTAXIS>>  
 ```````````````````````````````````````````````````````````````````````````````````````````````````````````````````````` 
  ``########`````##````````##``````````##`````````####````````##```##########````````#``````##``````###```##`````######`` 
  `##``````## ```##````````##`````````####````````##`##```````##```````##```````````###``````##````##`````##```##`````##` 
  ##````````##```##````````##````````##`##````````##``##``````##```````##``````````####```````#```##``````##```##``````## 
  ##````````##```##````````##```````##```##```````##```##`````##```````##`````````##`##```````##`##```````##````##``````` 
  ##````````##```##````````##``````##`````##``````##````##````##```````##````````##``###```````###````````##`````##`````` 
  ##````````##```##````````##``````##``````##`````##`````##```##```````##```````##````##```````###````````##``````###```` 
  ##````````##```##````````##`````##`````

In [57]:
can = qa.QA_fetch_stock_day_adv('000725', start='2017-06-01', end='2018-05-31').to_qfq()

In [58]:
raw = can.data.loc[:, ['open', 'close', 'high', 'low']].values

In [59]:
dates = can.data.date

In [60]:
from pyecharts import Grid, Bar, Line, Kline, Overlap

kline = Kline()
kline.add("日K", dates.tolist(), raw, is_datazoom_show=True)

overlap = Overlap()
overlap.add(kline)
overlap

In [61]:
mrg = []
mrg_cnt = []
mrg_date = []
raw = raw
raw_date = dates.tolist()
raw_idx = 0
mrg_idx = 0

while True:
    if raw_idx >= len(raw):
        break
    if raw_idx == 0:
        mrg.append(raw[raw_idx])
        mrg_date.append(raw_date[raw_idx])
        mrg_cnt.append(1)
        raw_idx += 1
        continue
    if (mrg[mrg_idx][2] >= raw[raw_idx][2] and mrg[mrg_idx][3] <= raw[raw_idx][3]) or (mrg[mrg_idx][2] <= raw[raw_idx][2] and mrg[mrg_idx][3] >= raw[raw_idx][3]):
        if mrg_idx == 0 or mrg[mrg_idx-1][2] <= mrg[mrg_idx][2]:
            mrg[mrg_idx][0] = max(mrg[mrg_idx][0], raw[raw_idx][0])
            mrg[mrg_idx][1] = max(mrg[mrg_idx][1], raw[raw_idx][1])
            mrg[mrg_idx][2] = max(mrg[mrg_idx][2], raw[raw_idx][2])
            mrg[mrg_idx][3] = max(mrg[mrg_idx][3], raw[raw_idx][3])
        else:
            mrg[mrg_idx][0] = min(mrg[mrg_idx][0], raw[raw_idx][0])
            mrg[mrg_idx][1] = min(mrg[mrg_idx][1], raw[raw_idx][1])
            mrg[mrg_idx][2] = min(mrg[mrg_idx][2], raw[raw_idx][2])
            mrg[mrg_idx][3] = min(mrg[mrg_idx][3], raw[raw_idx][3])
        mrg_date[mrg_idx] = raw_date[raw_idx]
        mrg_cnt[mrg_idx] += 1
        raw_idx += 1
        continue
    mrg.append(raw[raw_idx])
    mrg_date.append(raw_date[raw_idx])
    mrg_cnt.append(1)
    raw_idx += 1
    mrg_idx += 1

[(mrg_date[idx], mrg[idx][2], mrg[idx][3], mrg_cnt[idx]) for idx in range(len(mrg))]

[(Timestamp('2017-06-01 00:00:00'), 3.9100759481801033, 3.7909873406213186, 1),
 (Timestamp('2017-06-02 00:00:00'), 3.8703797456605074, 3.77113923936152, 1),
 (Timestamp('2017-06-05 00:00:00'), 3.9001518975502045, 3.8306835431409128, 1),
 (Timestamp('2017-06-06 00:00:00'), 3.86045569503061, 3.78106328999142, 1),
 (Timestamp('2017-06-07 00:00:00'), 3.9199999988100025, 3.7909873406213186, 1),
 (Timestamp('2017-06-08 00:00:00'), 4.039088606368788, 3.86045569503061, 1),
 (Timestamp('2017-06-12 00:00:00'), 4.049012656998686, 3.969620251959496, 2),
 (Timestamp('2017-06-13 00:00:00'), 4.009316454479091, 3.9398481000698, 1),
 (Timestamp('2017-06-16 00:00:00'), 3.8902278469203058, 3.8505316444007107, 3),
 (Timestamp('2017-06-19 00:00:00'), 3.9993924038491926, 3.919999998810003, 1),
 (Timestamp('2017-06-22 00:00:00'), 4.0688607582584835, 3.989468353219294, 3),
 (Timestamp('2017-06-23 00:00:00'), 4.019240505108989, 3.9398481000698, 1),
 (Timestamp('2017-06-26 00:00:00'), 4.187949365817268, 3.9894

In [62]:
kline = Kline()
kline.add("日K", mrg_date, mrg, is_datazoom_show=True, 
          mark_point=[{'coord':[mrg_date[-10], mrg[-10][2] * 1.03], 'name':'test'}], 
          mark_point_symbol='circle', mark_point_symbolsize=10)
kline
#overlap = Overlap()
#overlap.add(kline)
#overlap

In [63]:
def higher(x, y):
    return x[2] >= y[2] and x[3] >= y[3]

def lower(x, y):
    return x[2] <= y[2] and x[3] <= y[3]

end = []
last = 0
for idx, cell in enumerate(mrg):
    insert = False
    if idx < 2:
        continue
    if (higher(mrg[idx-1], mrg[idx-2]) and higher(mrg[idx-1], cell)):
        end.append([idx-1, 1, True])
        insert = True
    elif (lower(mrg[idx-1], mrg[idx-2]) and lower(mrg[idx-1], cell)):
        end.append([idx-1, 0, True])
        insert = True
    
    if insert and len(end)>1 and end[-1][1] == end[-2][1]:
        if ((end[-1][1] == 1 and mrg[end[-1][0]][2] > mrg[end[-2][0]][2]) or 
            (end[-1][1] == 0 and mrg[end[-1][0]][3] < mrg[end[-2][0]][3])):
            if not(idx-2 == last and mrg_cnt[last] + mrg_cnt[last+1] > 2 or idx-3 > last) and len(end) > 2:
                end.pop(-2)
            else:
                end[-2][2] = False
        else:
            end.pop()
            insert = False
    elif insert and not(idx-2 == last and mrg_cnt[last] + mrg_cnt[last+1] > 2 or idx-3 > last):
        end.pop()
        insert = False
    
    if insert:
        last = idx

[(item[0], str(mrg_date[item[0]]), 'top' if item[1] else 'bottom', 'valid' if item[2] else 'not') for item in end]

[(3, '2017-06-06 00:00:00', 'bottom', 'valid'),
 (10, '2017-06-22 00:00:00', 'top', 'not'),
 (14, '2017-06-30 00:00:00', 'top', 'valid'),
 (19, '2017-07-13 00:00:00', 'bottom', 'not'),
 (21, '2017-07-18 00:00:00', 'bottom', 'valid'),
 (28, '2017-08-01 00:00:00', 'top', 'valid'),
 (35, '2017-08-11 00:00:00', 'bottom', 'valid'),
 (43, '2017-08-28 00:00:00', 'top', 'not'),
 (47, '2017-09-06 00:00:00', 'top', 'valid'),
 (53, '2017-09-22 00:00:00', 'bottom', 'valid'),
 (60, '2017-10-16 00:00:00', 'top', 'not'),
 (69, '2017-11-01 00:00:00', 'top', 'not'),
 (73, '2017-11-08 00:00:00', 'top', 'valid'),
 (78, '2017-11-20 00:00:00', 'bottom', 'not'),
 (82, '2017-11-28 00:00:00', 'bottom', 'valid'),
 (86, '2017-12-05 00:00:00', 'top', 'not'),
 (90, '2017-12-12 00:00:00', 'top', 'valid'),
 (94, '2017-12-18 00:00:00', 'bottom', 'valid'),
 (98, '2017-12-22 00:00:00', 'top', 'valid'),
 (105, '2018-01-08 00:00:00', 'bottom', 'valid'),
 (109, '2018-01-17 00:00:00', 'top', 'not'),
 (111, '2018-01-23 00:

In [64]:
valid = [x for x in end if x[2]]
valid

[[3, 0, True],
 [14, 1, True],
 [21, 0, True],
 [28, 1, True],
 [35, 0, True],
 [47, 1, True],
 [53, 0, True],
 [73, 1, True],
 [82, 0, True],
 [90, 1, True],
 [94, 0, True],
 [98, 1, True],
 [105, 0, True],
 [111, 1, True],
 [118, 0, True],
 [122, 1, True],
 [127, 0, True],
 [129, 1, True],
 [137, 0, True],
 [141, 1, True],
 [143, 0, True],
 [147, 1, True],
 [151, 0, True],
 [153, 1, True],
 [159, 0, True]]

In [65]:

class Line(object):
    def __init__(self, start_idx, end_idx, ochl):
        self.start_idx = start_idx
        self.end_idx = end_idx
        self.is_bull = ochl[start_idx][2] < ochl[end_idx][2] and ochl[start_idx][3] < ochl[end_idx][3]
        self.start = ochl[start_idx][3] if self.is_bull else ochl[start_idx][2]
        self.end = ochl[end_idx][2] if self.is_bull else ochl[end_idx][3]
        self.high = self.end if self.is_bull else self.start
        self.low = self.start if self.is_bull else self.end
        span = self.high - self.low
        self.low_energy = self.low + span * 0.382 if self.is_bull else self.high - span * 0.382
        self.mid_energy = self.low + span * 0.5
        self.high_energy = self.low + span * 0.382 if not self.is_bull else self.high - span * 0.382
        self.first_target = self.high + span * 0.618 if self.is_bull else self.low - span * 0.618
        self.second_target = self.high + span * 1.618 if self.is_bull else self.low - span * 1.618
        self.valid = True
    
    def tostring(self):
        return '{} {} to {}'.format(self.start_idx, 'up' if self.is_bull else 'down', self.end_idx)


In [13]:
lines = [Line(a[0], b[0], mrg, mrg_date) for a, b in zip(valid[:-1], valid[1:])]

In [33]:
def find_lines(points):

    lines = [Line(points[0][0], points[1][0], mrg)]

    for idx, point in enumerate(points):
        if idx < 2:
            continue
        if point[1] == 0:
            lines = [l for l in lines if (mrg[point[0]][1] > l.low_energy and l.is_bull == True) or (mrg[point[0]][3] > l.second_target and l.is_bull == False)]
            starts = [l.start_idx for l in lines if l.is_bull == False and l.end > mrg[point[0]][3]]
            lines.extend([Line(x, point[0], mrg) for x in starts] + [Line(points[idx-1][0], point[0], mrg)])
        else:
            lines = [l for l in lines if (mrg[point[0]][1] < l.low_energy and l.is_bull == False) or (mrg[point[0]][2] < l.second_target and l.is_bull == True)]
            starts = [l.start_idx for l in lines if l.is_bull == True and l.end < mrg[point[0]][2]]
            lines.extend([Line(x, point[0], mrg) for x in starts] + [Line(points[idx-1][0], point[0], mrg)])

    return lines


In [66]:
[x.tostring() for x in find_lines(valid)]

['111 down to 118',
 '129 down to 137',
 '111 down to 143',
 '129 down to 143',
 '111 down to 151',
 '129 down to 151',
 '111 down to 151',
 '129 down to 151',
 '147 down to 151',
 '111 down to 159',
 '129 down to 159',
 '111 down to 159',
 '129 down to 159',
 '153 down to 159']

In [68]:
lines = find_lines(valid)
support = set()
pressure = set()

for l in lines:
    if l.is_bull:
        support.add((l.low_energy, '0.382 of {} up to {}'.format(l.start_idx, l.end_idx)))
        support.add((l.mid_energy, '0.5 of {} up to {}'.format(l.start_idx, l.end_idx)))
        support.add((l.high_energy, '0.618 of {} up to {}'.format(l.start_idx, l.end_idx)))
        pressure.add((l.high, 'high of {} up to {}'.format(l.start_idx, l.end_idx)))
        pressure.add((l.first_target, '1.618 of {} up to {}'.format(l.start_idx, l.end_idx)))
        pressure.add((l.second_target, '2.618 of {} up to {}'.format(l.start_idx, l.end_idx)))
    else:
        pressure.add((l.low_energy, '0.382 of {} down to {}'.format(l.start_idx, l.end_idx)))
        pressure.add((l.mid_energy, '0.5 of {} down to {}'.format(l.start_idx, l.end_idx)))
        pressure.add((l.high_energy, '0.618 of {} down to {}'.format(l.start_idx, l.end_idx)))
        support.add((l.low, 'low of {} down to {}'.format(l.start_idx, l.end_idx)))
        support.add((l.first_target, '1.618 of {} down to {}'.format(l.start_idx, l.end_idx)))
        support.add((l.second_target, '2.618 of {} down to {}'.format(l.start_idx, l.end_idx)))

sup = sorted(list(support), key=lambda x: x[0], reverse=True)
pre = sorted(list(pressure), key=lambda x: x[0])

print([s for s in sup if s[0] < raw[-1][1]])
print([p for p in pre if p[0] > raw[-1][1]])


[(4.10738, '1.618 of 111 down to 118'), (4.069880000000001, '1.618 of 153 down to 159'), (3.9941199999999997, '1.618 of 111 down to 143'), (3.7298800000000014, '2.618 of 153 down to 159'), (3.7237999999999984, '2.618 of 129 down to 137'), (3.546380000000001, '1.618 of 147 down to 151'), (3.3049199999999983, '2.618 of 129 down to 143'), (3.1676000000000006, '1.618 of 129 down to 159'), (3.07052, '1.618 of 129 down to 151'), (2.796800000000001, '1.618 of 111 down to 159'), (2.69972, '1.618 of 111 down to 151'), (2.5173799999999993, '2.618 of 111 down to 118'), (2.4563800000000007, '2.618 of 147 down to 151'), (2.3341199999999986, '2.618 of 111 down to 143'), (1.3676, '2.618 of 129 down to 159'), (1.2105199999999985, '2.618 of 129 down to 151'), (0.39680000000000026, '2.618 of 111 down to 159'), (0.23971999999999882, '2.618 of 111 down to 151')]
[(4.409880000000001, '0.618 of 153 down to 159'), (4.450000000000001, '0.5 of 153 down to 159'), (4.490120000000001, '0.382 of 153 down to 159'),

In [55]:
s = qa.QA_fetch_stock_min_adv('000005', start='2017-06-01', end='2018-05-30', frequence='60min').to_qfq()

C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:979: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_nested_tuple(tup)


In [56]:
s.data.tail(10)

code  open  high   low  close     volume  \
datetime            code                                                 
2018-05-28 14:00:00 000005  000005  3.66  3.67  3.65   3.65   535000.0   
2018-05-28 15:00:00 000005  000005  3.64  3.68  3.59   3.68  3241700.0   
2018-05-29 10:30:00 000005  000005  3.63  3.65  3.60   3.61  1402600.0   
2018-05-29 11:30:00 000005  000005  3.61  3.62  3.58   3.58  1229100.0   
2018-05-29 14:00:00 000005  000005  3.59  3.63  3.57   3.62  1187600.0   
2018-05-29 15:00:00 000005  000005  3.61  3.62  3.59   3.60  1184500.0   
2018-05-30 10:30:00 000005  000005  3.52  3.58  3.49   3.51  3548000.0   
2018-05-30 11:30:00 000005  000005  3.52  3.52  3.48   3.50  1875900.0   
2018-05-30 14:00:00 000005  000005  3.50  3.52  3.49   3.52   753000.0   
2018-05-30 15:00:00 000005  000005  3.52  3.53  3.38   3.40  4991400.0   

                            preclose            datetime        date  \
datetime            code                                               
2018-05-28 14:00:00 000005      3.67 2018-05-28 14:00:00  2018-05-28   
2018-05-28 15:00:00 000005      3.65 2018-05-28 15:00:00  2018-05-28   
2018-05-29 10:30:00 000005      3.68 2018-05-29 10:30:00  2018-05-29   
2018-05-29 11:30:00 000005      3.61 2018-05-29 11:30:00  2018-05-29   
2018-05-29 14:00:00 000005      3.58 2018-05-29 14:00:00  2018-05-29   
2018-05-29 15:00:00 000005      3.62 2018-05-29 15:00:00  2018-05-29   
2018-05-30 10:30:00 000005      3.60 2018-05-30 10:30:00  2018-05-30   
2018-05-30 11:30:00 000005      3.51 2018-05-30 11:30:00  2018-05-30   
2018-05-30 14:00:00 000005      3.50 2018-05-30 14:00:00  2018-05-30   
2018-05-30 15:00:00 000005      3.52 2018-05-30 15:00:00  2018-05-30   

                            high_limit  low_limit  
datetime            code                           
2018-05-28 14:00:00 000005        4.04       3.30  
2018-05-28 15:00:00 000005        4.02       3.29  
2018-05-29 10:30:00 000005        4.05       3.31  
2018-05-29 11:30:00 000005        3.97       3.25  
2018-05-29 14:00:00 000005        3.94       3.22  
2018-05-29 15:00:00 000005        3.98       3.26  
2018-05-30 10:30:00 000005        3.96       3.24  
2018-05-30 11:30:00 000005        3.86       3.16  
2018-05-30 14:00:00 000005        3.85       3.15  
2018-05-30 15:00:00 000005        3.87       3.17